# Fraudfinder - Inference Demo (New Feature Store)

## Overview

This series of labs are updated upon [FraudFinder](https://github.com/googlecloudplatform/fraudfinder) repository which builds a end-to-end real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.


### Objective

This notebook demonstrates a critical step in a real-time fraud detection system: **making a prediction on a new, incoming transaction.** We'll simulate a real-world scenario where a transaction is received and we need to quickly determine if it's fraudulent.

To do this, we'll use a deployed machine learning model on a **Vertex AI Endpoint**. However, a model needs more than just the raw transaction data to make an accurate prediction; it needs **features**. These are the data points that give the model context about the transaction, the customer, and the terminal involved.

This is where the **Vertex AI Feature Store** comes in. The Feature Store provides low-latency access to pre-calculated features. In this notebook, we will:

1. **Simulate a new transaction** by pulling a sample transaction from a Pub/Sub topic.
2. **Enrich the transaction data** by fetching the latest customer and terminal features from the Vertex AI Feature Store.
3. **Construct a prediction request** with the combined transaction data and features.
4. **Send the request to the Vertex AI Endpoint** to get a real-time fraud prediction.

This entire process mirrors a production-level inference pipeline, showcasing how to leverage Vertex AI services to build a powerful and responsive fraud detection system.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

### Import libraries and define constants

#### Libraries
Here, we'll import the necessary libraries for this notebook.

In [ ]:
from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1 import (
    FeatureOnlineStoreServiceClient,
)
from google.cloud.aiplatform_v1.types import (
    feature_online_store_service as feature_online_store_service_pb2,
)

In [ ]:
# Vertex AI SDK
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

### Define helper methods

In [ ]:
def read_from_sub(project_id, subscription_name, messages=10):
    """
    Read messages from a Pub/Sub subscription
    Args:
        project_id: project ID
        subscription_name: the name of a Pub/Sub subscription in your project
        messages: number of messages to read
    Returns:
        msg_data: list of messages in your Pub/Sub subscription as a Python dictionary
    """

    import ast

    from google.api_core import retry
    from google.cloud import pubsub_v1

    subscriber = pubsub_v1.SubscriberClient()
    subscription_path = subscriber.subscription_path(
        project_id, subscription_name
    )

    # Wrap the subscriber in a 'with' block to automatically call close() to
    # close the underlying gRPC channel when done.
    with subscriber:
        # The subscriber pulls a specific number of messages. The actual
        # number of messages pulled may be smaller than max_messages.
        response = subscriber.pull(
            subscription=subscription_path,
            max_messages=messages,
            retry=retry.Retry(deadline=300),
        )

        if len(response.received_messages) == 0:
            print("no messages")
            return

        ack_ids = []
        msg_data = []
        for received_message in response.received_messages:
            msg = ast.literal_eval(
                received_message.message.data.decode("utf-8")
            )
            msg_data.append(msg)
            ack_ids.append(received_message.ack_id)

        # Acknowledges the received messages so they will not be sent again.
        subscriber.acknowledge(subscription=subscription_path, ack_ids=ack_ids)

        print(
            f"Received and acknowledged {len(response.received_messages)} messages from {subscription_path}."
        )

        return msg_data

### Retrieve the Deployed Model Endpoint

Before we can get a prediction, we need to connect to our deployed model. In Vertex AI, deployed models are exposed as **Endpoints**. An endpoint provides a URL that you can send prediction requests to. The following code retrieves the endpoint for our trained fraud detection model. We filter by the `display_name` to ensure we get the correct one.

In [ ]:
endpoints = vertex_ai.Endpoint.list(
    filter=f"display_name={ENDPOINT_NAME}",  # optional: filter by specific endpoint name
    order_by="update_time",
)

ENDPOINT_ID = endpoints[-1].name
print(ENDPOINT_ID)

In [ ]:
print(ENDPOINT_ID)
from google.cloud import aiplatform as aiplatform

# Instantiate Vertex AI Endpoint object
endpoint_obj = aiplatform.Endpoint(ENDPOINT_ID)

### Simulate an Incoming Transaction

In a real-world system, new transactions would be continuously streaming in. To simulate this, we'll read a single transaction from the `ff-tx-sub` Pub/Sub subscription. This subscription receives the same raw transaction data that our real-time feature engineering pipeline processes. The `read_from_sub` helper function will pull one message for us to use in our prediction request.


In [ ]:
import time

messages_tx = "no messages"
while messages_tx == "no messages":
    messages_tx = read_from_sub(
        project_id=PROJECT_ID, subscription_name="ff-tx-sub", messages=1
    )
    print(messages_tx)
    time.sleep(5)  # Sleep for 5 seconds

messages_tx

### Connect to the Vertex AI Feature Store

Now that we have our model endpoint, we need a way to get the features for our incoming transaction. This is where the **Vertex AI Feature Store** is essential. It's designed for low-latency, real-time feature lookups, which is exactly what's needed in a fraud detection scenario.

To connect to the Feature Store, we'll instantiate a `FeatureOnlineStoreServiceClient`. This client will allow us to query the online store for the latest feature values.

In [ ]:
API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

data_client = FeatureOnlineStoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

### Define the Feature Store Lookup Function

To make it easier to retrieve features, we'll define a helper function called `fs_features_lookup`. This function will take the Feature Store ID, the type of feature we're looking for (e.g., `customers` or `terminals`), and an entity ID (e.g., a specific customer ID) as input.

Inside the function, we'll construct the full ID of the **Feature View** we want to query. A Feature View is a logical grouping of features from a single entity type. The function then uses the `data_client` to call the `fetch_feature_values` method, which returns the latest feature values for the given entity ID. This function is the core of our real-time feature enrichment process.

In [ ]:
import pprint

pp = pprint.PrettyPrinter(compact=True)


def fs_features_lookup(ff_feature_store, features_type, features_key):

    FEATURE_VIEW_ID = f"fv_fraudfinder_{features_type}"
    FEATURE_VIEW_FULL_ID = f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{ff_feature_store}/featureViews/{FEATURE_VIEW_ID}"

    features_map = {}

    print(FEATURE_VIEW_FULL_ID)

    try:
        fe_continuous_data = data_client.fetch_feature_values(
            request=feature_online_store_service_pb2.FetchFeatureValuesRequest(
                feature_view=FEATURE_VIEW_FULL_ID,
                data_key=feature_online_store_service_pb2.FeatureViewDataKey(
                    key=features_key
                ),
                data_format=feature_online_store_service_pb2.FeatureViewDataFormat.PROTO_STRUCT,
            )
        )
        features_map.update(
            {k: v for k, v in fe_continuous_data.proto_struct.items()}
        )
    except Exception as exp:
        print(f"Requested entity {features_key} was not found")
    return features_map

### Test the Feature Lookup

Before we build the full prediction request, let's test our `fs_features_lookup` function. We'll get a recent `entity_id` from our streaming features table in BigQuery and use it to fetch the corresponding customer features from the Feature Store. This helps us verify that our connection to the Feature Store is working and see what the feature data looks like.

In [ ]:
%%bigquery customer_strm_record --project {PROJECT_ID}
SELECT * FROM tx.t_customers_streaming_features
ORDER BY feature_timestamp DESC
LIMIT 1

In [ ]:
customer_strm_record

In [ ]:
customer_key = customer_strm_record["entity_id"][0]
print(f"entity_id={customer_key}")

In [ ]:
customer_features = fs_features_lookup(
    FEATURESTORE_ID, "customers", customer_key  # customer_key
)
pp.pprint(customer_features)

In [ ]:
%%bigquery terminal_strm_record --project {PROJECT_ID}
SELECT * FROM tx.v_terminals_features
ORDER BY feature_timestamp DESC
LIMIT 1

In [ ]:
terminal_key = terminal_strm_record["entity_id"][0]
print(f"entity_id={customer_key}")
terminal_features = fs_features_lookup(
    FEATURESTORE_ID, "terminals", terminal_key
)  # Change key values
pp.pprint(terminal_features)

### Putting It All Together: Building the Prediction Request

Now we'll perform the full, end-to-end process for a single transaction:

1.  **Get a transaction:** We'll use the transaction we read from Pub/Sub earlier.
2.  **Set default feature values:** We start with a dictionary of default values for all our features. This is a good practice to ensure the model receives a complete feature vector, even if a feature lookup fails.
3.  **Add transaction amount:** We add the `TX_AMOUNT` from our simulated transaction to the payload.
4.  **Lookup and add customer features:** We use our `fs_features_lookup` function to get the latest features for the customer in the transaction and add them to the payload.
5.  **Lookup and add terminal features:** We do the same for the terminal.
6.  **Send for prediction:** Finally, we send the complete payload to our model endpoint's `predict` method.

The result will be a real-time fraud prediction for our incoming transaction, enriched with the latest features from our Feature Store.

In [ ]:
import pprint

pp = pprint.PrettyPrinter(compact=True)

# Payload for manual test:
# payload_json = {
#     "TX_ID": "61210be0744c43232990152d3eb2c2deb6035d8b",
#     "TX_TS": "2025-09-06 17:27:51",
#     "CUSTOMER_ID": "7389471951168361",
#     "TERMINAL_ID": "45087784",
#     "TX_AMOUNT": 32.77
# }

default_features = {
    "customer_id_avg_amount_14day_window": 0,
    "customer_id_avg_amount_15min_window": 0,
    "customer_id_avg_amount_1day_window": 0,
    "customer_id_avg_amount_30min_window": 0,
    "customer_id_avg_amount_60min_window": 0,
    "customer_id_avg_amount_7day_window": 0,
    "customer_id_nb_tx_14day_window": 0,
    "customer_id_nb_tx_7day_window": 0,
    "customer_id_nb_tx_15min_window": 0,
    "customer_id_nb_tx_1day_window": 0,
    "customer_id_nb_tx_30min_window": 0,
    "customer_id_nb_tx_60min_window": 0,
    "terminal_id_avg_amount_15min_window": 0,
    "terminal_id_avg_amount_30min_window": 0,
    "terminal_id_avg_amount_60min_window": 0,
    "terminal_id_nb_tx_14day_window": 0,
    "terminal_id_nb_tx_15min_window": 0,
    "terminal_id_nb_tx_1day_window": 0,
    "terminal_id_nb_tx_30min_window": 0,
    "terminal_id_nb_tx_60min_window": 0,
    "terminal_id_nb_tx_7day_window": 0,
    "terminal_id_risk_14day_window": 0,
    "terminal_id_risk_1day_window": 0,
    "terminal_id_risk_7day_window": 0,
}

# Reading 1-st message
payload_json = messages_tx[0]

payload = default_features
payload["tx_amount"] = payload_json["TX_AMOUNT"]

# look up the customer features from New Vertex AI Feature Store
customer_features = fs_features_lookup(
    FEATURESTORE_ID, "customers", payload_json["CUSTOMER_ID"]
)
# print the customer features from Vertex AI Feature Store
print("-------------------------------------------------------")
print("customer_features:")
pp.pprint(customer_features)

# look up the treminal features from New Vertex AI Feature Store
terminal_features = fs_features_lookup(
    FEATURESTORE_ID, "terminals", payload_json["TERMINAL_ID"]
)
print("-------------------------------------------------------")
print("terminal features:")
pp.pprint(terminal_features)

# Add customer features to payload
payload.update(customer_features)

# Add terminal features to payload
payload.update(terminal_features)

# del payload["feature_timestamp"]

print("-------------------------------------------------------")
print("[Payload to be sent to Vertex AI endpoint]")
pp.pprint(payload)
print("-------------------------------------------------------")

result = endpoint_obj.predict(instances=[payload])

print("-------------------------------------------------------")
pp.pprint(f"[Prediction result]: {result}")
print("-------------------------------------------------------")